8. Load the MNIST data (introduced in Chapter 3), and split it into a training set, a
validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation,
and 10,000 for testing). Then train various classifiers, such as a Random
Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine
them into an ensemble that outperforms them all on the validation set, using a
soft or hard voting classifier. Once you have found one, try it on the test set. How
much better does it perform compared to the individual classifiers?  
<br><br>
9. Run the individual classifiers from the previous exercise to make predictions on
the validation set, and create a new training set with the resulting predictions:
each training instance is a vector containing the set of predictions from all your
classifiers for an image, and the target is the image’s class. Train a classifier on
this new training set. Congratulations, you have just trained a blender, and
together with the classifiers they form a stacking ensemble! Now let’s evaluate the
ensemble on the test set. For each image in the test set, make predictions with all
your classifiers, then feed the predictions to the blender to get the ensemble’s predictions.
How does it compare to the voting classifier you trained earlier?

# 8

In [1]:
import numpy as np
import pandas as pd
from numpy.random import randn

import matplotlib.pyplot as plt
import seaborn as sns
import dataprep as dp

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

import warnings
#warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)

In [3]:
y[5]

2

In [5]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=2021)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=2021)

In [6]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
from sklearn.ensemble import VotingClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=2021)
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=2021)
svm_clf = SVC(random_state=2021)

voting_clf = VotingClassifier(
    estimators=[('et', et_clf), ('rf', rf_clf), ('svc', svm_clf)],
    voting='hard')

estimators = [rf_clf, et_clf, svm_clf]
for estimator in estimators:
    estimator.fit(X_train, y_train)

In [9]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9669, 0.9705, 0.9772]

In [11]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('et', ExtraTreesClassifier(random_state=2021)),
                             ('rf', RandomForestClassifier(random_state=2021)),
                             ('svc', SVC(random_state=2021))])

In [12]:
voting_clf.score(X_val, y_val)

0.9722

In [15]:
voting_clf2 = VotingClassifier(
    estimators=[('et', et_clf), ('rf', rf_clf), ('svc', SVC(probability=True, random_state=2021))],
    voting='soft')
voting_clf2.fit(X_train, y_train)

VotingClassifier(estimators=[('et', ExtraTreesClassifier(random_state=2021)),
                             ('rf', RandomForestClassifier(random_state=2021)),
                             ('svc', SVC(probability=True, random_state=2021))],
                 voting='soft')

In [16]:
voting_clf2.score(X_val, y_val)

0.9773

# 9

In [17]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [18]:
X_val_predictions

array([[6., 6., 6.],
       [9., 9., 9.],
       [4., 4., 4.],
       ...,
       [9., 9., 9.],
       [8., 8., 8.],
       [3., 3., 3.]], dtype=float32)

In [19]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [20]:
rnd_forest_blender.oob_score_

0.9742

In [21]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)
    
y_pred = rnd_forest_blender.predict(X_test_predictions)
accuracy_score(y_test, y_pred)

0.9772